## Instructions

- what you're looking at is a Jupyter Notebook, I'm using it as a way to execute snippets of Python code in a web browser
- the script uses basic features of a Python library called "Pandas" to do the data manipulation, I assume there's an equivalent in R
- you can run the script by clicking into the code block below pressing the small play button above
  - the script will read whatever CSV you have in the input.csv file, and write the processed output to an output.csv file (see file browser to left for these)
  - after runnning the script, it'll print out a snippet of the processed output at the bottom of the page
  - to see the updated output, you may need to hit "Refresh file list" in the file browser side panel

In [ ]:
import pandas as pd

def is_a_buy_txn(txn):
    sum_buy = txn["sum_buy"]
    sum_sell = txn["sum_sell"]
    return sum_buy > sum_sell

def is_a_sell_txn(txn):
    return not is_a_buy_txn(txn)

def calc_sell_volume(txn):
    # if inventory decreased from one day to the next, consider it a sale and return the delta
    if txn["net_inventory"] < txn["prev_net_inventory"]:
        return txn["prev_net_inventory"] - txn["net_inventory"]
    
    # otherwise, assume we don't sell units
    return 0

def append_calcs_for_txns_for_date(txns_for_date):
    cumulative_net_inventory_for_day = 0

    for i, txn in txns_for_date.iterrows():
        if is_a_buy_txn(txn):
            txns_for_date.at[i, "net_inventory"] = cumulative_net_inventory_for_day + txn["sum_buy"]
            cumulative_net_inventory_for_day = txns_for_date.at[i, "net_inventory"]
            continue

        if is_a_sell_txn(txn):
            # we assume we start with 0 inventory
            # if the sale would yield negative inventory, sell 100% of inventory, and reset the net inventory back to 0
            # otherwise, we just decrement the net inventory by the amount we sold
            next_net_inventory = cumulative_net_inventory_for_day - txn["sum_sell"]
            if next_net_inventory < 0:
                txns_for_date.at[i, "net_inventory"] = 0
                cumulative_net_inventory_for_day = 0
            else:
                txns_for_date.at[i, "net_inventory"] = next_net_inventory
                cumulative_net_inventory_for_day = txns_for_date.at[i, "net_inventory"]
            continue

    txns_for_date["prev_net_inventory"] = txns_for_date["net_inventory"].shift(1)
    txns_for_date["sell_volume"] = txns_for_date.apply(calc_sell_volume, axis=1)

    txns_for_date["cost_of_buys"] = txns_for_date["sum_buy"] * txns_for_date["avg_price"]
    txns_for_date["cum_cost"] = txns_for_date["cost_of_buys"].cumsum()

    txns_for_date["revenue_from_sells"] = txns_for_date["sell_volume"] * txns_for_date["avg_price"]
    txns_for_date["cum_revenue"] = txns_for_date["revenue_from_sells"].cumsum()
    txns_for_date = txns_for_date.fillna(0)
    txns_for_date["profit"] = txns_for_date["cum_revenue"] - txns_for_date["cum_cost"]
    return txns_for_date

# read the file input.csv into a dataframe
input_df = pd.read_csv("./input.csv").fillna(0).assign(net_inventory=0)

working_df = pd.DataFrame(columns=input_df.columns)
for (date, txns_for_date) in input_df.groupby("date"):
    ret = append_calcs_for_txns_for_date(txns_for_date)
    working_df = pd.concat([working_df, ret])
    
# stripping out the intermediate columns added to the working dataframe
output_df = working_df[input_df.columns.append(pd.Index(["cum_cost", "cum_revenue", "profit"]))]

# write the processed dataframe into the output file out.csv
# out.csv will contain the additional columns of "cum_cost", "cum_revenue" and "profit"
output_df.to_csv("./out.csv")

from IPython.display import display, HTML

# input
display(HTML(input_df[input_df["date"] == "1/9/12"].to_html()))

# output
display(HTML(output_df[output_df["date"] == "1/9/12"].to_html()))